In [1]:
# 데이터 구성: Series, DataFrame
import pandas as pd
import numpy as np


#탐색적 분석을 위한 패키지
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale,minmax_scale,robust_scale
from scipy.stats import shapiro

#ARIMA
from statsmodels.tsa.arima_model import ARIMA



#matplotlib 이용한 background 그래프 실행 및 결과 확인 
%matplotlib inline


#export_graphviz: 나무구조 생성 및 저장
from sklearn.tree import export_graphviz
#graphviz: 나무 구조 시각화(.dot 확장자 파일 불러오기 등)
import graphviz


# 데이터 분할: train,test
from sklearn.model_selection import train_test_split

#데이터 scaling => 거리기반 알고리즘은 필수!!!
from sklearn.preprocessing import StandardScaler
# 데이터 분할: train,test
from sklearn.model_selection import train_test_split
# 분류 DT
from sklearn.tree import DecisionTreeClassifier
#분류  RF
from sklearn.ensemble import RandomForestClassifier
# 분류 GB
from sklearn.ensemble import GradientBoostingClassifier
# 분류 SVM
from sklearn.svm import SVC
# 분류 KNN
from sklearn.neighbors import KNeighborsClassifier
# 분류 NN
from sklearn.neural_network import MLPClassifier

# 최적모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 모델 성능 평가 
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
# 한글 폰트 적용
font_path = '/usr/share/fonts/opentype/noto/NotoSerifCJK-ExtraLight.ttc'
font_name = matplotlib.font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font', family = font_name)

In [3]:
# 데이터 구성하기(MFG_TAT)
df_tat = pd.read_csv("/home/pirl/빅데이터 프로젝트/cleansing/outlier/MFG_TAT_NULL_COL_OUT.csv", engine="python",encoding='EUC-KR')
df_tat.head()

,JUDGE,LOT_ID,PANEL_ID,TIME_1BLACK,TIME_2AG,TIME_3BUS,TIME_4DIELEC,TIME_5MGO,TIME_6PHOS,TIME_7PASTE,...,TAT_3BUS_2FIRE,TAT_4DIELEC_FIRE,TAT_5MGO,TAT_6PHOS_1R_1DISPENSER,TAT_6PHOS_1R_2DRY,TAT_6PHOS_2G_1DISPENSER,TAT_6PHOS_2G_2DRY,TAT_6PHOS_3B_1DISPENSER,TAT_6PHOS_4FIRE,TAT_8AGING
0,양품,LOT-101,PNL-01,2019-04-01T00:01,2019-04-01T05:21,2019-04-01T11:01,2019-04-06T13:41,2019-04-11T03:41,2019-04-03T20:20,2019-04-11T12:41,...,6900,6447,530,76,333,88,396,86,7140,10074
1,양품,LOT-101,PNL-02,2019-04-01T00:45,2019-04-01T06:05,2019-04-01T11:45,2019-04-06T14:25,2019-04-11T04:25,2019-04-03T21:04,2019-04-11T13:25,...,6693,6600,530,80,438,82,381,84,6956,10075
2,양품,LOT-101,PNL-03,2019-04-01T01:29,2019-04-01T06:49,2019-04-01T12:29,2019-04-06T15:09,2019-04-11T05:09,2019-04-03T21:48,2019-04-11T14:09,...,6747,6597,530,80,420,92,396,86,6840,10645
3,양품,LOT-101,PNL-04,2019-04-01T02:13,2019-04-01T07:33,2019-04-01T13:13,2019-04-06T15:53,2019-04-11T05:53,2019-04-03T22:32,2019-04-11T14:53,...,7137,6420,530,78,429,78,399,90,6720,10076
4,양품,LOT-101,PNL-05,2019-04-01T02:58,2019-04-01T08:18,2019-04-01T13:58,2019-04-06T16:38,2019-04-11T06:38,2019-04-03T23:17,2019-04-11T15:38,...,7527,6453,540,76,363,82,393,84,7110,10074


In [4]:
# 데이터 구성하기(MFG_TAT)
df_mc = pd.read_csv("/home/pirl/빅데이터 프로젝트/cleansing/outlier/MFG_MACHINE_NULL_COL_OUT.csv", engine="python",encoding='EUC-KR')
df_mc.head()

,JUDGE,LOT_ID,PANEL_ID,TIME_1BLACK,TIME_2AG,TIME_3BUS,TIME_4DIELEC,TIME_5MGO,TIME_6PHOS,TIME_7PASTE,...,MC_4DIELEC_1FIRE,MC_5MGO,MC_6PHOS_1R_1DISPENSER,MC_6PHOS_1R_2DRY,MC_6PHOS_2G_1DISPENSER,MC_6PHOS_2G_2DRY,MC_6PHOS_3B_1DISPENSER,MC_6PHOS_4FIRE,MC_7PASTE,MC_8AGING
0,양품,LOT-101,PNL-01,2019-04-01T00:01,2019-04-01T05:21,2019-04-01T11:01,2019-04-06T13:41,2019-04-11T03:41,2019-04-03T20:20,2019-04-11T12:41,...,3호기,1호기,1호기,2호기,2호기,2호기,1호기,3호기,1호기,2호기
1,양품,LOT-101,PNL-02,2019-04-01T00:45,2019-04-01T06:05,2019-04-01T11:45,2019-04-06T14:25,2019-04-11T04:25,2019-04-03T21:04,2019-04-11T13:25,...,3호기,1호기,1호기,2호기,2호기,2호기,1호기,3호기,1호기,2호기
2,양품,LOT-101,PNL-03,2019-04-01T01:29,2019-04-01T06:49,2019-04-01T12:29,2019-04-06T15:09,2019-04-11T05:09,2019-04-03T21:48,2019-04-11T14:09,...,3호기,1호기,1호기,2호기,2호기,2호기,1호기,3호기,1호기,2호기
3,양품,LOT-101,PNL-04,2019-04-01T02:13,2019-04-01T07:33,2019-04-01T13:13,2019-04-06T15:53,2019-04-11T05:53,2019-04-03T22:32,2019-04-11T14:53,...,3호기,1호기,1호기,2호기,2호기,2호기,1호기,3호기,1호기,2호기
4,양품,LOT-101,PNL-05,2019-04-01T02:58,2019-04-01T08:18,2019-04-01T13:58,2019-04-06T16:38,2019-04-11T06:38,2019-04-03T23:17,2019-04-11T15:38,...,3호기,1호기,1호기,2호기,2호기,2호기,1호기,3호기,1호기,2호기


In [12]:
# 데이터 구성하기(MFG_TAT)
df_process = pd.read_csv("/home/pirl/빅데이터 프로젝트/cleansing/outlier/MFG_PROCESS_NULL_COL_OUT.csv", engine="python",encoding='EUC-KR')
df_process.head()

,JUDGE,LOT_ID,PANEL_ID,TIME_1BLACK,TIME_2AG,TIME_3BUS,TIME_4DIELEC,TIME_5MGO,TIME_6PHOS,TIME_7PASTE,...,PHOS_G_DRY_TEMP_HOOD_START,PHOS_G_DRY_TEMP_HOOD_MAX,PHOS_FIRE_TEMP_ANEAL_CH01,PHOS_FIRE_TEMP_ANEAL_CH02,PHOS_FIRE_TEMP_ANEAL_CH03,PHOS_FIRE_TEMP_ANEAL_CH04,PHOS_FIRE_EXHAUST_HEAT1,PHOS_FIRE_EXHAUST_HEAT2,PHOS_FIRE_EXHAUST_HEAT3,PHOS_FIRE_EXHAUST_HEAT4
0,양품,LOT-101,PNL-01,2019-04-01T00:01,2019-04-01T05:21,2019-04-01T11:01,2019-04-06T13:41,2019-04-11T03:41,2019-04-03T20:20,2019-04-11T12:41,...,82.8,84.0,443,419,392,372,4115.0,3973.0,4432,2506.0
1,양품,LOT-101,PNL-02,2019-04-01T00:45,2019-04-01T06:05,2019-04-01T11:45,2019-04-06T14:25,2019-04-11T04:25,2019-04-03T21:04,2019-04-11T13:25,...,80.3,81.8,445,423,395,374,3642.0,4097.0,4499,2573.0
2,양품,LOT-101,PNL-03,2019-04-01T01:29,2019-04-01T06:49,2019-04-01T12:29,2019-04-06T15:09,2019-04-11T05:09,2019-04-03T21:48,2019-04-11T14:09,...,81.2,82.2,444,421,393,372,4063.0,3972.0,4460,2502.0
3,양품,LOT-101,PNL-04,2019-04-01T02:13,2019-04-01T07:33,2019-04-01T13:13,2019-04-06T15:53,2019-04-11T05:53,2019-04-03T22:32,2019-04-11T14:53,...,83.0,84.0,444,420,393,372,4131.0,4011.0,4466,2522.0
4,양품,LOT-101,PNL-05,2019-04-01T02:58,2019-04-01T08:18,2019-04-01T13:58,2019-04-06T16:38,2019-04-11T06:38,2019-04-03T23:17,2019-04-11T15:38,...,80.1,82.0,443,420,392,372,3632.0,4073.0,4482,2584.0


In [6]:
tat = df_tat[['PANEL_ID','TAT_3BUS_1DEVELOP']]
tat.head()

,PANEL_ID,TAT_3BUS_1DEVELOP
0,PNL-01,416
1,PNL-02,372
2,PNL-03,368
3,PNL-04,416
4,PNL-05,380


In [16]:
tat2 = df_tat[['PANEL_ID','TAT_2AG_2RTD']]

In [7]:
mc = df_mc[['PANEL_ID','MC_3BUS_1DEVELOP']]
mc.head()

,PANEL_ID,MC_3BUS_1DEVELOP
0,PNL-01,1호기
1,PNL-02,1호기
2,PNL-03,1호기
3,PNL-04,1호기
4,PNL-05,1호기


In [17]:
mc2 = df_mc[['PANEL_ID','MC_2AG_2RTD']]

In [18]:
merge3 = tat2.merge(mc2)
merge3.head()

,PANEL_ID,TAT_2AG_2RTD,MC_2AG_2RTD
0,PNL-01,74,1호기
1,PNL-01,74,1호기
2,PNL-01,74,1호기
3,PNL-01,74,2호기
4,PNL-01,74,1호기


In [19]:
merge3 = merge3.drop('PANEL_ID', axis=1)
merge3.groupby('MC_2AG_2RTD').agg('std')

,TAT_2AG_2RTD
MC_2AG_2RTD,
1호기,2.488493
2호기,2.488800


In [21]:
# merge3 = merge3.drop('PANEL_ID', axis=1)
merge3.groupby('MC_2AG_2RTD').agg('mean')

,TAT_2AG_2RTD
MC_2AG_2RTD,
1호기,70.558016
2호기,70.557751


In [13]:
process = df_process[['PANEL_ID', 'BUS_DEVELOP_TEMP_TANK2']]
process.head()

,PANEL_ID,BUS_DEVELOP_TEMP_TANK2
0,PNL-01,30.753
1,PNL-02,30.800
2,PNL-03,30.694
3,PNL-04,30.691
4,PNL-05,31.032


In [14]:
merge2 = process.merge(mc)
merge2

,PANEL_ID,BUS_DEVELOP_TEMP_TANK2,MC_3BUS_1DEVELOP
0,PNL-01,30.753,1호기
1,PNL-01,30.753,1호기
2,PNL-01,30.753,2호기
3,PNL-01,30.753,2호기
4,PNL-01,30.753,1호기
...,...,...,...
792827,PNL-20,30.805,2호기
792828,PNL-20,30.805,1호기
792829,PNL-20,30.805,2호기
792830,PNL-20,30.805,1호기


In [15]:
merge2 = merge2.drop('PANEL_ID', axis=1)
merge2.groupby('MC_3BUS_1DEVELOP').agg('std')

,BUS_DEVELOP_TEMP_TANK2
MC_3BUS_1DEVELOP,
1호기,0.355482
2호기,0.355410


In [22]:
# merge2 = merge2.drop('PANEL_ID', axis=1)
merge2.groupby('MC_3BUS_1DEVELOP').agg('mean')

,BUS_DEVELOP_TEMP_TANK2
MC_3BUS_1DEVELOP,
1호기,30.979160
2호기,30.979148


In [11]:
merge = tat.merge(mc)
merge

,PANEL_ID,TAT_3BUS_1DEVELOP,MC_3BUS_1DEVELOP
0,PNL-01,416,1호기
1,PNL-01,416,1호기
2,PNL-01,416,2호기
3,PNL-01,416,2호기
4,PNL-01,416,1호기
...,...,...,...
792827,PNL-20,332,2호기
792828,PNL-20,332,1호기
792829,PNL-20,332,2호기
792830,PNL-20,332,1호기


In [9]:
merge = merge.drop('PANEL_ID', axis=1)

In [10]:
merge.groupby('MC_3BUS_1DEVELOP').agg('std')

,TAT_3BUS_1DEVELOP
MC_3BUS_1DEVELOP,
1호기,21.138281
2호기,21.138743


In [23]:
merge.groupby('MC_3BUS_1DEVELOP').agg('mean')

,TAT_3BUS_1DEVELOP
MC_3BUS_1DEVELOP,
1호기,370.946931
2호기,370.951579
